# README.

This notebook is the entrypoint for Azure ML enabled training.
In its essence, it connects to Azure ML, makes sure that everything is ready over there, and starts the training.
To that end, this notebook gathers all necessary sourcecodes in a temp-folder, which will be pushed to Azure ML for training.

# Imports.

In [1]:
%reload_ext autoreload
%autoreload 2

import os
import shutil
#import urllib

from pathlib import Path


from azureml.core import Workspace, Dataset, Experiment, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import MpiConfiguration
from azureml.train.dnn import TensorFlow

# idk why this is not working
from src.constants import REPO_DIR
from src.train_util import copy_dir
from src.config import CONFIG, DATASET_MODE_MOUNT, DATASET_MODE_DOWNLOAD

# Setting screws

In [2]:
dataset_name = "anon-depthmap-95k"
experiment_name = "q3-cnndepthmap-resnet-height-95k"
#tags = {} Q: DO I NEED THIS HERE?

# Create temp folder and copy code.

Here you have to be very precise, which code to copy.
And most importantly, which code NOT to copy.

In [3]:
#new
code_dir = Path("src")
temp_path = Path("temp_train")
copy_dir(src=code_dir, tgt=temp_path, glob_pattern='*.py')
copy_dir(src=REPO_DIR / "src/common", tgt=temp_path / "temp_common", glob_pattern='*/*.py', should_touch_init=True)

2021-04-27 11:02:12,871 - INFO - Creating temp folder - /mnt/resource/batch/tasks/shared/LS_root/mounts/clusters/jzcomp/code/Users/jziegler/cgm-ml/src/models/CNNDepthMap/CNNDepthMap-height/q3-cnndepthmap-resnet-height/src/train_util.py: line 10
2021-04-27 11:02:12,926 - INFO - Copying to temp_train the following files: [PosixPath('src/config.py'), PosixPath('src/constants.py'), PosixPath('src/model.py'), PosixPath('src/preprocessing.py'), PosixPath('src/train.py'), PosixPath('src/train_util.py'), PosixPath('src/utils.py'), PosixPath('src/__init__.py')] - /mnt/resource/batch/tasks/shared/LS_root/mounts/clusters/jzcomp/code/Users/jziegler/cgm-ml/src/models/CNNDepthMap/CNNDepthMap-height/q3-cnndepthmap-resnet-height/src/train_util.py: line 18
2021-04-27 11:02:14,932 - INFO - Creating temp folder - /mnt/resource/batch/tasks/shared/LS_root/mounts/clusters/jzcomp/code/Users/jziegler/cgm-ml/src/models/CNNDepthMap/CNNDepthMap-height/q3-cnndepthmap-resnet-height/src/train_util.py: line 10
2021-

# Connect to azure workspace.

Make sure that you have a config.json file with the keys subscription_id, resource_group, and cgm-ml-dev. Either here (not so nice) or in a parent folder (okay but not perfect), or in the root folder of this repo (way to go).

In [4]:
workspace = Workspace.from_config()
workspace

2021-04-27 11:02:23,699 - INFO - Found the config file in: /config.json - /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/core/workspace.py: line 287
2021-04-27 11:02:23,874 - INFO - cc9cf4a3-7f24-47d2-b769-cfbe66db26e8 - CacheDriver:Cached token is expired at 2021-03-16 11:12:11.656852.  Refreshing - /anaconda/envs/azureml_py36/lib/python3.6/site-packages/adal/log.py: line 114
2021-04-27 11:02:23,875 - INFO - cc9cf4a3-7f24-47d2-b769-cfbe66db26e8 - TokenRequest:Getting a new token from a refresh token - /anaconda/envs/azureml_py36/lib/python3.6/site-packages/adal/log.py: line 114
2021-04-27 11:02:24,130 - INFO - cc9cf4a3-7f24-47d2-b769-cfbe66db26e8 - CacheDriver:Returning token refreshed after expiry. - /anaconda/envs/azureml_py36/lib/python3.6/site-packages/adal/log.py: line 114


UserErrorException: UserErrorException:
	Message: You are currently logged-in to 006dabd7-456d-465b-a87f-f7d557e319c8 tenant. You don't have access to 9b5bbfae-d5d1-4aae-a2ca-75159c0c887d subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='cgm-2019-subscription', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to 006dabd7-456d-465b-a87f-f7d557e319c8 tenant. You don't have access to 9b5bbfae-d5d1-4aae-a2ca-75159c0c887d subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = \n [SubscriptionInfo(subscription_name='cgm-2019-subscription', subscription_id='9b82ecea-6780-4b85-8acf-d27d7902

# Get the experiment.

- You should always arrange all your runs in an experiment.
- Create at least one experiment per sprint.
- Make sure that the name of the experiment reflects the sprint number.
- On top of that you could also add other tokens to the name. For example network architecture, dataset name, and/or targets.

In [5]:
experiment = Experiment(workspace=workspace, name=experiment_name)
experiment

NameError: name 'workspace' is not defined

# Find/create a compute target.

Connects to a compute cluster on Azure ML.
If the compute cluster does not exist, it will be created.

Note: Usually computer clusters autoscale. This means that new nodes are created when necessary. And unused VMs will be shut down.

In [ ]:
cluster_name = "gpu-cluster"

# Compute cluster exists. Just connect to it.
try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print("Found existing compute target.")

# Compute cluster does not exist. Create one.    
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6', 
        max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

# Get the dataset for training.

Here you specify which dataset to use.

Note: Double check on Azure ML that you are using the right one.

In [ ]:
dataset = workspace.datasets[dataset_name]
dataset

# Push the training source code to Azure.

Creates an estimator (a template for a compute cluster node) and pushes it to the compute cluster.

In [ ]:
script_params = {f"--{k}": v for k, v in CONFIG.items()}
script_params

In [ ]:
curated_env_name = "cgm-env"

ENV_EXISTS = True
if ENV_EXISTS:
    cgm_env = Environment.get(workspace=workspace, name=curated_env_name)
else:
    cgm_env = Environment.from_conda_specification(name=curated_env_name, file_path=REPO_DIR / "environment_train.yml")
    cgm_env.docker.enabled = True
    cgm_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'
    # cgm_env.register(workspace)  # Please be careful not to overwrite existing environments

In [ ]:
if CONFIG.DATASET_MODE == DATASET_MODE_MOUNT:
    dataset_argument = dataset.as_named_input('cgm_dataset').as_mount()
elif CONFIG.DATASET_MODE == DATASET_MODE_DOWNLOAD:
    dataset_argument = dataset.as_named_input('cgm_dataset').as_download()
else:
    raise Exception("Please specify DATASET_MODE")

In [ ]:
# Create the ScriptRunConfig
script_run_config = ScriptRunConfig(source_directory=temp_path,
                                    compute_target=compute_target,
                                    script='train.py',
                                    arguments=[dataset_argument] + [str(item) for sublist in script_params.items() for item in sublist],
                                    environment=cgm_env,
)

# Set compute target.
script_run_config.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(config=script_run_config, tags=tags)

# Show run.
run

In [ ]:
#old TensorFlow.get_supported_versions()

In [ ]:
# old
# Specify pip packages here.
#pip_packages = [
#    "azureml-dataprep[fuse,pandas]",
#    "glob2",
#    "opencv-python",
#    "matplotlib",
#    "tensorflow-addons==0.11.2",
#    "bunch==1.0.1",
#]
#
#script_params = {
#    "--split_seed": 666,
#    "--target_size": "180x240",
#    "--epochs": 1000,
#    "--batch_size": 64,
#    "--normalization_value": 7.5,
#    "--res_blocks": "1,1,1,1",
#    "--dropouts": "0.2,0.3,0.4,0.5",
#    "--comment": "ResNet on 95K data. With dropout."
#}
#
## Create the estimator.
#estimator = TensorFlow(
#    source_directory=temp_path,
#    compute_target=compute_target,
#    entry_script="train.py",
#    use_gpu=True,
#    framework_version="2.0",
#    inputs=[dataset.as_named_input("dataset").as_mount()],
#    pip_packages=pip_packages,
#    script_params=script_params
#)
#
## Set compute target.
#estimator.run_config.target = compute_target
#
## Run the experiment.
#run = experiment.submit(estimator)
#
## Show outputs.
#run

# Delete temp folder.

After all code has been pushed to Azure ML, the temp folder will be removed.

In [ ]:
shutil.rmtree(temp_path)

## 